In [73]:
import json
import numpy as np
import os
import random
import pylab

from PIL import Image, ImageDraw
from collections import defaultdict

from matplotlib import pyplot as plt
from matplotlib import animation as animation

### Visualizing glimpses

In [81]:
def create_gif(img, pred, label, locs, num_glimpses, patch_size, resize_factor, out_file):
    # create draw object
    gif_image = img.copy()
    line_draw = ImageDraw.Draw(gif_image)

    frames = []
    for i in range(len(locs)):
        # get color for current glimpse
        point = locs[i]

        if i != 0:
            # All frames share the drawing of the line connecting glimpses
            line_draw.line((*prev_point, *point), fill="white")

        # In each frame only one glimpse will be visible, 
        frame = gif_image.copy()
        frame_draw = ImageDraw.Draw(frame)

        # draw center glimpse point
        frame_draw.point(point, fill="red")

        # draw an small circle around the center glimpse point (for better visibility)
        radius = 5
        shape = [(point[0]-radius, point[1]-radius), (point[0]+radius, point[1]+radius)]
        frame_draw.ellipse(shape, outline="red")

        # draw the area covered by the glimpse
        radius = patch_size/2 * resize_factor
        shape = [(point[0]-radius, point[1]-radius), (point[0]+radius, point[1]+radius)]
        frame_draw.rectangle(shape, outline="red")

        # update previous point
        prev_point = point
        
        # save frame
        frames.append(frame)

    # Set the initial frame
    fig, ax = plt.subplots()
    im = ax.imshow(frames[0], animated=True)
    ax.set_title(f"Prediction: {pred} | GT: {label}")
    ax.axis(False)

    def update(i):
        im.set_array(frames[i])
        return im

    # Create the animation (GIF)
    animation_fig = animation.FuncAnimation(
        fig, update, frames=len(frames), interval=400, blit=False, repeat_delay=200)
    animation_fig.save(out_file + ".gif")

    return frames


def visualize_glimpses(sample, patch_size, num_glimpses, save_gif=False, out_file=None):
    pylab.ioff()
    # load image and normalize
    img = np.array(sample["pixel_values"])
    img = (img * 0.3081) + 0.1307
    img = np.repeat(img, 3, axis=0)
    img = np.moveaxis(img, 0, 2)
    img = Image.fromarray(np.uint8(img*255))
    w, h = img.size
    
    pred, label = sample["pred"], sample["label"]

    # Resize to be visible
    resize_factor = 5
    img = img.resize((w*resize_factor, h*resize_factor))
    w, h = img.size

    # convert glimpse locations to pixel space
    locs = [[(i + 1)/2 * w, (j + 1)/2 * h] for i,j in sample["locs"]]

    # create draw object
    draw_img = img.copy()
    draw = ImageDraw.Draw(draw_img)

    for i in range(len(locs)):
        # draw center glimpse point
        point = locs[i]
        draw.point(point, fill="red")

        if i != 0:
            # Line joining past and current glimpse
            draw.line((*prev_point, *point), fill="white")

        # draw an small circle around the center glimpse point (for better visibility)
        radius = 5
        shape = [(point[0]-radius, point[1]-radius), (point[0]+radius, point[1]+radius)]
        draw.ellipse(shape, outline="red")

        # draw the area covered by the glimpse
        radius = patch_size/2 * resize_factor
        shape = [(point[0]-radius, point[1]-radius), (point[0]+radius, point[1]+radius)]
        draw.rectangle(shape, outline="red")

        # update previous point
        prev_point = point
    
    fig, ax = plt.subplots()
    ax.imshow(draw_img)
    ax.set_title(f"Prediction: {pred} | GT: {label}")
    ax.axis(False)
    fig.savefig(out_file + ".png")
    
    if save_gif:
        create_gif(img, pred, label, locs, num_glimpses, patch_size, resize_factor, out_file)
    
    pylab.ion()
    return draw_img


In [82]:
#ckpt_dir = "/data/users/jmorales/model_files/RAM/baseline"
#ckpt_dir = "/data/users/jmorales/model_files/RAM/GPT2"
#ckpt_dir = "/data/users/jmorales/model_files/RAM/GTrXL"
ckpt_dir = "/data/users/jmorales/model_files/RAM/TrXL"

res_file = [i for i in os.listdir(ckpt_dir) if "results.json" in i][0]
res_file = os.path.join(ckpt_dir, res_file)

with open(res_file, "r") as f:
    results = json.load(f)

In [83]:
good = [i for i in results if i["pred"] == i["label"]]
bad = [i for i in results if i["pred"] != i["label"]]

In [84]:
for samples, label in zip([good, bad], ["correct", "wrong"]):
    samples = random.sample(samples, 10)
    for i, sample in enumerate(samples):
        visualize_glimpses(
            sample, 
            patch_size=8, 
            num_glimpses=6, 
            save_gif=True, 
            out_file=f"../plots/Experiment1/TrXL_{label}_{i}"
            )